<a href="https://colab.research.google.com/github/nesrinechemkhi/Eniac_Discounts/blob/main/Eniac_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_rows', 1000)
url = "https://drive.google.com/file/d/1k-TVAtN1NTnG0bjs-44aS-ZDio4Js7Ul/view?usp=drive_link" # brands.csv
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
df_brands = pd.read_csv(path)
url = "https://drive.google.com/file/d/1mGCeISUhuqrTLH5xG0zmNOPgJ4IxJPld/view?usp=drive_link" # orderlines.csv
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
df_orderlines = pd.read_csv(path)
url = "https://drive.google.com/file/d/1yZdpAB9RiD0H3nVqIqryR_VyQHCFNEz_/view?usp=drive_link" # orders.csv
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
df_orders = pd.read_csv(path)
url = "https://drive.google.com/file/d/14jNM6cSZXf-6lsayF6zL7vPWCJbiQkMi/view?usp=drive_link" # products.csv
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
df_products = pd.read_csv(path)
# create table copies
df_orderlines_copy = df_orderlines.copy()
df_orders_copy = df_orders.copy()
df_products_copy = df_products.copy()
df_brands_copy = df_brands.copy()

In [ ]:
df_products_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19326 entries, 0 to 19325
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sku       19326 non-null  object
 1   name      19326 non-null  object
 2   desc      19319 non-null  object
 3   price     19280 non-null  object
 4   in_stock  19326 non-null  int64 
 5   type      19326 non-null  object
dtypes: int64(1), object(5)
memory usage: 906.0+ KB


In [ ]:
# convert dates
df_orderlines_copy['date'] = pd.to_datetime(df_orderlines_copy['date'])
df_orders_copy['created_date'] = pd.to_datetime(df_orders_copy['created_date'])
# convert product category types
df_products_copy['type'] = df_products_copy['type'].astype(str)
# drop non-needed columns
df_orderlines_copy.drop(columns=['product_id'], inplace=True)
df_products_copy.drop(columns=['promo_price'], inplace=True)
# drop duplicates
df_orderlines_copy.drop_duplicates()
df_products_copy.drop_duplicates()
df_brands_copy.drop_duplicates()
df_orders_copy.drop_duplicates()
# drop non-needed states
indices_to_drop = df_orders_copy[(df_orders_copy['state'] == 'Shopping Basket') | (df_orders_copy['state'] == 'Cancelled') ].index
orders_copy = df_orders_copy.drop(indices_to_drop).copy()
# eliminate corrupted rows
unit_price_mask = df_orderlines_copy['unit_price'].str.count('\.') > 1
orderlines_df = df_orderlines_copy.loc[~unit_price_mask].copy()
orderlines_df['unit_price'] = pd.to_numeric(orderlines_df['unit_price'])

unit_price_mask_2 = df_products_copy['price'].str.count('\.') > 1
products_df = df_products_copy.loc[~unit_price_mask_2].copy()
products_df['price'] = pd.to_numeric(products_df['price'])

num_missing = orders_copy['total_paid'].isna()
orders_df_copy = orders_copy.loc[~num_missing].copy()

num_missing2 = orderlines_df['unit_price'].isna()
orderlines_df_copy = orderlines_df.loc[~num_missing2].copy()


In [ ]:
orders_df_copy.value_counts('state')

,count
state,
Completed,46605
Place Order,40883
Pending,14374


In [ ]:
orderlines_df_copy.info()

<class 'pandas.core.frame.DataFrame'>
Index: 257814 entries, 0 to 293982
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   id                257814 non-null  int64         
 1   id_order          257814 non-null  int64         
 2   product_quantity  257814 non-null  int64         
 3   sku               257814 non-null  object        
 4   unit_price        257814 non-null  float64       
 5   date              257814 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(3), object(1)
memory usage: 13.8+ MB


In [ ]:
orders_df_copy.info()

<class 'pandas.core.frame.DataFrame'>
Index: 101862 entries, 1 to 226908
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   order_id      101862 non-null  int64         
 1   created_date  101862 non-null  datetime64[ns]
 2   total_paid    101862 non-null  float64       
 3   state         101862 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 3.9+ MB


In [ ]:
# keep only rows in common between orders and orderlines
orders_in_o_df = orders_df_copy['order_id']
orderlines_connected = orderlines_df_copy.loc[orderlines_df_copy['id_order'].isin(orders_in_o_df)].copy()

orders_in_ol_df = orderlines_connected['id_order']
orders_connected = orders_df_copy.loc[orders_df_copy['order_id'].isin(orders_in_ol_df)]
# clean the products table
existing_products = df_products_copy['sku']
corrupted_filter = ~(orderlines_connected['sku'].isin(existing_products))

corrupted_orders = orderlines_connected.loc[corrupted_filter, 'id_order']
clean_orders = orders_connected.loc[~(orders_connected['order_id'].isin(corrupted_orders))]

clean_orderlines = orderlines_connected.loc[~(orderlines_connected['id_order'].isin(corrupted_orders))]

In [ ]:
orders_connected.value_counts('state')

,count
state,
Completed,44113
Place Order,24427
Pending,13593


In [ ]:
clean_orderlines.info()

<class 'pandas.core.frame.DataFrame'>
Index: 255024 entries, 0 to 293982
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   id                255024 non-null  int64         
 1   id_order          255024 non-null  int64         
 2   product_quantity  255024 non-null  int64         
 3   sku               255024 non-null  object        
 4   unit_price        255024 non-null  float64       
 5   date              255024 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(3), object(1)
memory usage: 13.6+ MB


In [ ]:
clean_orders.info()

<class 'pandas.core.frame.DataFrame'>
Index: 185312 entries, 0 to 226908
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   order_id      185312 non-null  int64         
 1   created_date  185312 non-null  datetime64[ns]
 2   total_paid    185312 non-null  float64       
 3   state         185312 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 7.1+ MB


In [ ]:
# merge orderlines with products
merged_df = clean_orderlines.merge(products_df,
                            left_on="sku",
                            right_on="sku",
                            how = "inner")
# add revenue column
merged_df["revenues"] = merged_df["unit_price"] * merged_df["product_quantity"]

In [ ]:
# import categories file
url = "https://drive.google.com/file/d/1Tnw1hRN2xs2aXZrfOlFSS0efSxtKDyO9/view?usp=sharing"
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
df_categories = pd.read_csv(path)


In [ ]:
merged_df_2 = merged_df.merge(df_categories,
                            left_on="type",
                            right_on="type",
                            how = "inner")

In [ ]:
merged_df_3 = merged_df_2.merge(clean_orders,
                            left_on="id_order",
                            right_on="order_id",
                            how = "inner")

In [ ]:
merged_df_3['price'] = merged_df_3['price'].astype(float)
merged_df_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200398 entries, 0 to 200397
Data columns (total 17 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   id                200398 non-null  int64         
 1   id_order          200398 non-null  int64         
 2   product_quantity  200398 non-null  int64         
 3   sku               200398 non-null  object        
 4   unit_price        200398 non-null  float64       
 5   date              200398 non-null  datetime64[ns]
 6   name              200398 non-null  object        
 7   desc              200386 non-null  object        
 8   price             200278 non-null  float64       
 9   in_stock          200398 non-null  int64         
 10  type              200398 non-null  object        
 11  revenues          200398 non-null  float64       
 12  Category  Name    200398 non-null  object        
 13  order_id          200398 non-null  int64         
 14  crea

In [ ]:

indices_to_drop_3 = merged_df_3[(merged_df_3['state'] == 'Shopping Basket') | (merged_df_3['state'] == 'Cancelled') ].index
merged_df_4 = merged_df_3.drop(indices_to_drop_3)

indices_to_drop_4 = merged_df_4[merged_df_4['price'].isnull()].index
merged_df_5 = merged_df_4.drop(indices_to_drop_4)
merged_df_5.info()


<class 'pandas.core.frame.DataFrame'>
Index: 90774 entries, 5 to 200397
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   id                90774 non-null  int64         
 1   id_order          90774 non-null  int64         
 2   product_quantity  90774 non-null  int64         
 3   sku               90774 non-null  object        
 4   unit_price        90774 non-null  float64       
 5   date              90774 non-null  datetime64[ns]
 6   name              90774 non-null  object        
 7   desc              90770 non-null  object        
 8   price             90774 non-null  float64       
 9   in_stock          90774 non-null  int64         
 10  type              90774 non-null  object        
 11  revenues          90774 non-null  float64       
 12  Category  Name    90774 non-null  object        
 13  order_id          90774 non-null  int64         
 14  created_date      90774 no

In [ ]:
# actual price from products table minus the unit price that was paid for from the orderlines
merged_df_5["price_diff"] = merged_df_5["price"] - merged_df_5["unit_price"]
# the difference divided by the actual price
merged_df_5["discount_percent"] = (merged_df_5["price_diff"] / merged_df_5["price"]) * 100
merged_df_5.sort_values(by="discount_percent", ascending=False)

,id,id_order,product_quantity,sku,unit_price,date,name,desc,price,in_stock,type,revenues,Category Name,order_id,created_date,total_paid,state,price_diff,discount_percent
186742,1615921,512498,5,MOP0107,1.60,2018-02-16 08:54:42,Mophie Powerstation 4000mAh Battery Plus Mini ...,external battery capacity 4000mAh output volta...,69.95,0,1515,8.00,battery_related,512498,2018-02-16 09:02:03,46.94,Completed,68.35,97.71
186702,1615825,512451,5,MOP0107,1.60,2018-02-16 08:35:43,Mophie Powerstation 4000mAh Battery Plus Mini ...,external battery capacity 4000mAh output volta...,69.95,0,1515,8.00,battery_related,512451,2018-02-16 08:51:32,52.93,Completed,68.35,97.71
32923,1203340,334415,1,BEL0145-A,25.47,2017-03-12 22:13:33,Open - Belkin Switch + Motion WeMO Wi-Fi contr...,Kit + motion switch for sensor devices via App.,999.94,0,1298,25.47,miscellaneous,334415,2017-03-12 22:14:52,102.45,Pending,974.47,97.45
187082,1616752,512862,2,MOP0109,2.00,2018-02-16 12:47:23,Mophie Powerstation Mini Universal 4000mAh Bat...,external battery capacity 4000mAh output volta...,69.95,0,1515,4.00,battery_related,512862,2018-02-16 13:01:31,20.97,Completed,67.95,97.14
173118,1574841,496421,1,PIE0034-A,11.79,2018-01-22 22:35:46,Open - Piece Internal Battery iPhone 6,Replacement internal battery for Apple iPhone 6,349.90,0,21485407,11.79,pieces,496421,2018-01-22 22:39:07,15.78,Completed,338.11,96.63
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30406,1197459,331788,1,IFX0074,24.95,2017-03-03 23:11:45,iFixit piece Earpiece Speaker iPhone 6,Handset internal speaker for iPhone 6,7.99,1,21485407,24.95,pieces,331788,2017-03-04 20:02:13,64.92,Pending,-16.96,-212.27
46333,1233154,347233,1,IFX0074,24.95,2017-04-22 12:20:43,iFixit piece Earpiece Speaker iPhone 6,Handset internal speaker for iPhone 6,7.99,1,21485407,24.95,pieces,347233,2017-04-25 10:53:24,64.91,Completed,-16.96,-212.27
74945,1297936,379925,1,IFX0074,24.95,2017-07-20 12:27:58,iFixit piece Earpiece Speaker iPhone 6,Handset internal speaker for iPhone 6,7.99,1,21485407,24.95,pieces,379925,2017-07-20 12:33:12,31.94,Completed,-16.96,-212.27
82712,1316185,388586,1,IFX0074,24.95,2017-08-11 09:29:25,iFixit piece Earpiece Speaker iPhone 6,Handset internal speaker for iPhone 6,7.99,1,21485407,24.95,pieces,388586,2017-08-11 10:27:30,66.93,Completed,-16.96,-212.27


In [ ]:
merged_df_6 = merged_df_5.loc[(merged_df_5["discount_percent"] >= 0) & (merged_df_5["discount_percent"] <= 100)]
merged_df_6.sort_values(by="discount_percent", ascending=False)

,id,id_order,product_quantity,sku,unit_price,date,name,desc,price,in_stock,type,revenues,Category Name,order_id,created_date,total_paid,state,price_diff,discount_percent
186742,1615921,512498,5,MOP0107,1.60,2018-02-16 08:54:42,Mophie Powerstation 4000mAh Battery Plus Mini ...,external battery capacity 4000mAh output volta...,69.95,0,1515,8.00,battery_related,512498,2018-02-16 09:02:03,46.94,Completed,68.35,97.71
186702,1615825,512451,5,MOP0107,1.60,2018-02-16 08:35:43,Mophie Powerstation 4000mAh Battery Plus Mini ...,external battery capacity 4000mAh output volta...,69.95,0,1515,8.00,battery_related,512451,2018-02-16 08:51:32,52.93,Completed,68.35,97.71
32923,1203340,334415,1,BEL0145-A,25.47,2017-03-12 22:13:33,Open - Belkin Switch + Motion WeMO Wi-Fi contr...,Kit + motion switch for sensor devices via App.,999.94,0,1298,25.47,miscellaneous,334415,2017-03-12 22:14:52,102.45,Pending,974.47,97.45
187082,1616752,512862,2,MOP0109,2.00,2018-02-16 12:47:23,Mophie Powerstation Mini Universal 4000mAh Bat...,external battery capacity 4000mAh output volta...,69.95,0,1515,4.00,battery_related,512862,2018-02-16 13:01:31,20.97,Completed,67.95,97.14
173118,1574841,496421,1,PIE0034-A,11.79,2018-01-22 22:35:46,Open - Piece Internal Battery iPhone 6,Replacement internal battery for Apple iPhone 6,349.90,0,21485407,11.79,pieces,496421,2018-01-22 22:39:07,15.78,Completed,338.11,96.63
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28994,1194323,330353,1,APP1706,439.00,2017-02-28 14:11:00,Apple Watch Series 2 Aluminum 38mm Rose Gold C...,Apple Watch 38 mm with GPS dual-core processor...,439.00,0,24885185,439.00,watch,330353,2017-02-28 14:05:02,439.00,Place Order,0.00,0.00
111368,1396927,421648,1,APP2516,39.00,2017-11-03 20:10:55,Silicon Case Cover Apple iPhone 8/7 Red Rose,Ultrathin silicone case and microfiber premium...,39.00,0,11865403,39.00,iphone_accessories,421648,2017-11-03 20:10:55,39.00,Place Order,0.00,0.00
189899,1624234,516153,1,PIE0009,14.99,2018-02-22 18:21:15,Internal Battery for iPhone 4S,Replacement AC Adapter for Apple iPhone 4S.,14.99,1,21485407,14.99,pieces,516153,2018-02-22 18:45:02,18.98,Completed,0.00,0.00
111355,1396902,421631,1,APP2164,149.00,2017-11-03 19:39:47,"Apple iPad Leather Sleeve Case Pro 10.5 ""Blue ...",Leather case with full protection and slot Pen...,149.00,0,12635403,149.00,cases,421631,2017-11-03 19:42:02,1339.81,Completed,0.00,0.00


In [ ]:
merged_df_6.info()

<class 'pandas.core.frame.DataFrame'>
Index: 86360 entries, 5 to 200397
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   id                86360 non-null  int64         
 1   id_order          86360 non-null  int64         
 2   product_quantity  86360 non-null  int64         
 3   sku               86360 non-null  object        
 4   unit_price        86360 non-null  float64       
 5   date              86360 non-null  datetime64[ns]
 6   name              86360 non-null  object        
 7   desc              86356 non-null  object        
 8   price             86360 non-null  float64       
 9   in_stock          86360 non-null  int64         
 10  type              86360 non-null  object        
 11  revenues          86360 non-null  float64       
 12  Category  Name    86360 non-null  object        
 13  order_id          86360 non-null  int64         
 14  created_date      86360 no

In [ ]:
# product categories
merged_df_6.loc[merged_df_6["type"] == "1282"].sample(10)

,id,id_order,product_quantity,sku,unit_price,date,name,desc,price,in_stock,type,revenues,Category Name,order_id,created_date,total_paid,state,price_diff,discount_percent
85487,1322343,391560,2,APP0900,765.59,2017-08-22 12:43:41,Apple Mac mini Core i5 26GHz | 8GB RAM | 1TB HDD,PC Mac mini Core i5 8GB 1TB 26GHz (MGEN2YP / A).,805.59,0,1282,1531.18,apple_mac,391560,2017-08-22 12:54:12,1531.19,Completed,40.00,4.97
17789,1168446,318600,1,PAC1921,318.99,2017-02-01 01:15:29,"Second hand - Apple iMac 20 ""Core 2 Duo 24GHz ...",IMac used 20 inch Core 2 Duo 24GHz | 3GB RAM |...,1199.00,0,1282,318.99,apple_mac,318600,2017-02-01 01:18:30,330.98,Completed,880.01,73.40
10379,1149699,311007,1,APP0899,512.99,2017-01-17 14:48:37,Apple Mac mini Core i5 14GHz | 4GB RAM | 500GB...,PC Mac mini Core i5 4GB 500GB 14GHz (MGEM2YP /...,555.59,0,1282,512.99,apple_mac,311007,2017-01-17 14:52:32,512.99,Completed,42.60,7.67
23039,1180259,324057,1,APP0900,745.99,2017-02-14 13:24:32,Apple Mac mini Core i5 26GHz | 8GB RAM | 1TB HDD,PC Mac mini Core i5 8GB 1TB 26GHz (MGEN2YP / A).,805.59,0,1282,745.99,apple_mac,324057,2017-02-14 13:24:32,745.99,Place Order,59.60,7.40
23058,1180305,324074,1,APP0900,774.99,2017-02-14 13:40:23,Apple Mac mini Core i5 26GHz | 8GB RAM | 1TB HDD,PC Mac mini Core i5 8GB 1TB 26GHz (MGEN2YP / A).,805.59,0,1282,774.99,apple_mac,324074,2017-02-14 13:40:23,774.99,Place Order,30.60,3.80
109822,1393202,419934,1,APP0899,525.60,2017-10-30 17:12:05,Apple Mac mini Core i5 14GHz | 4GB RAM | 500GB...,PC Mac mini Core i5 4GB 500GB 14GHz (MGEM2YP /...,555.59,0,1282,525.60,apple_mac,419934,2017-10-30 23:36:53,540.58,Completed,29.99,5.40
39395,1217859,341205,1,APP0900,739.99,2017-03-31 12:15:01,Apple Mac mini Core i5 26GHz | 8GB RAM | 1TB HDD,PC Mac mini Core i5 8GB 1TB 26GHz (MGEN2YP / A).,805.59,0,1282,739.99,apple_mac,341205,2017-03-31 12:19:46,739.99,Pending,65.60,8.14
131624,1453422,446155,1,APP0911,965.59,2017-11-26 20:40:26,Apple Mac mini Core i5 26GHz | 16GB RAM | 1TB HDD,PC Mac mini Core i5 26GHz 16GB 1TB (MGEN2YP / A).,1039.00,0,1282,965.59,apple_mac,446155,2017-11-26 20:45:24,965.59,Completed,73.41,7.07
51663,1244868,354356,1,APP1581,949.00,2017-05-10 00:03:15,"Apple MacBook Air 13 ""i5 16 Ghz | 8GB RAM | 12...",laptop MacBook Air 13 inch i5 16GHz 8GB RAM 12...,1099.00,0,1282,949.00,apple_mac,354356,2017-05-10 00:11:22,1121.98,Pending,150.00,13.65
48535,1238298,351063,1,APP1581,949.00,2017-04-29 14:03:06,"Apple MacBook Air 13 ""i5 16 Ghz | 8GB RAM | 12...",laptop MacBook Air 13 inch i5 16GHz 8GB RAM 12...,1099.00,0,1282,949.00,apple_mac,351063,2017-04-29 14:03:06,949.00,Place Order,150.00,13.65
